In [11]:
# Load longwave and shortwave datasets
loCld_lw_flux_cld = FBCT_flux.toa_lw_cldtyp_mon.sel(press=slice(0.0,1.0))
loCld_lw_flux_clr = FBCT_flux.toa_lw_clr_mon
loCld_sw_flux_cld = FBCT_flux.toa_sw_cldtyp_mon.sel(press=slice(0.0,1.0))
loCld_sw_flux_clr = FBCT_flux.toa_sw_clr_mon

# Get net DOWNWARD flux and then the climatological mean
loCld_net_flux_all = (-loCld_sw_flux_cld - loCld_lw_flux_cld).mean(dim='time')
loCld_net_flux_clr = (-loCld_sw_flux_clr - loCld_lw_flux_clr).mean(dim='time')

# Now, downsample the data
loCld_net_flux_all_loRes = loCld_net_flux_all.interp(latitude=new_lat, longitude=new_lon)
loCld_net_flux_clr_loRes = loCld_net_flux_clr.interp(latitude=new_lat, longitude=new_lon)

In [16]:
# Load the low cloud fraction
loCld_frac = FBCT_flux.cldarea_cldtyp_mon.sel(press=slice(0.0,1.0)) / 100

# Next, downsample
loCld_frac_loRes = loCld_frac.interp(latitude=new_lat, longitude=new_lon)

loCld_frac_anom = get_anomaly(loCld_frac_loRes)
loCld_frac_enso = composite(loCld_frac_anom)

In [39]:
# Finally, get monthly low-cloud radiative anomaly for enso yrs *** (R_pt - R_clr)*f'_pt ***
loCld_cre = get_mean_across_bins(loCld_net_flux_all_loRes,loCld_net_flux_clr_loRes,loCld_frac_enso)

In [43]:
def get_ceres_cre():
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(2,figsize=(12,6), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
    fig.tight_layout(h_pad=6)

    # Give each plot gridlines, coastlines, and block out continents
    for i in np.arange(0,2,1):
        ax[i].gridlines()
        ax[i].coastlines()
        ax[i].add_feature(cartopy.feature.LAND,zorder=100,facecolor='k',edgecolor='k')

    # Plot Trends
    loCld_cre.sel(nino34='nino').plot(ax=ax[0],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           cbar_kwargs={'shrink': 0.9},vmin=-5,vmax=5)
    ax[0].set_title('CERES Low Cloud CRE Anomaly - El Nino')
    
    loCld_cre.sel(nino34='nina').plot(ax=ax[1],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           cbar_kwargs={'shrink': 0.9})#,vmin=-.1,vmax=.1)
    ax[1].set_title('CERES Low Cloud CRE Anomaly - La Nina')